# Not Really Stable Workflow for DraCor on Docker

Cf. relevant notebooks:

* https://github.com/dracor-org/dracor-notebooks/blob/docker/docker/local-dracor-with-docker.ipynb
* https://github.com/dracor-org/vebidracor/blob/main/vebidracor-workflow.ipynb


```
services:
  api:
    image: ingoboerner/dracor-api:v0.86.3_local
    ports:
      - "8080:8080"
    depends_on:
      - metrics
      - fuseki
  metrics:
    image: ingoboerner/dracor-metrics:v1.2.1
    ports:
      - "8030:8030"
  frontend:
    image: ingoboerner/dracor-frontend:v1.4.3_local
    ports:
      - "8088:80"
    depends_on:
      - api
  fuseki:
    image: "stain/jena-fuseki"
    environment:
      - ADMIN_PASSWORD=qwerty
      - FUSEKI_DATASET_1=dracor
    ports:
      - "3030:3030"
    expose:
      - "3030"
```

Run
`docker-compose -f docker-compose.empty.yml up` in cmd prompt or similar

Go to http://localhost:8088/

Should be an empty DraCor instance.

Functions to use the API:
Helper function, might become obsolete if pydracor is ready.

In [1]:
%pip install requests

In [2]:
# import libraries json and requests
import json
import requests

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True

    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://localhost:8088/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"]
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]

    else:
        #nothing set
        request = request_url = apibase + "info"

    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

# 1. Setup an empty corpus in local instance

In [4]:
new_corpus_name = "ai"
new_corpus_title = "AI Drama Corpus"

#needed for authorization
from requests.auth import HTTPBasicAuth

#Username of the local instance
usr = "admin"
#Password of the admin user
pwd = "rXEVM26cN66ARa97864BG7h5"   # this changes every time, you find it in the cmd prompt chronology 

#construct the payload
metadata = {
  "name": new_corpus_name,
  "title": new_corpus_title
}

#url of the corpora endpoint
corpora_endpoint_url = "http://localhost:8088/api/corpora"

#send the POST request using library requests
r = requests.post(corpora_endpoint_url, json = metadata, auth=HTTPBasicAuth(usr, pwd))

if r.status_code == 200:
    print("Success!" + " http://localhost:8088/" + new_corpus_name)
else:
    print(r.status_code)

Success! http://localhost:8088/ai


In [ ]:
#check if successful
get(method="corpora", parse_json=True)

# Load plays from my own repository

### Get URLS

In [ ]:
'''
import PyGithub

g = github.Github()
repo = g.get_repo("lucagiovannini7/artificial-dramas/ezdrama_format/gpt/xmls")

contents = repo.get_contents("")
list_urls = []

for file_content in contents:
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    else:
        list_urls.append(file_content)

for url in list_urls:
    print(url.name)
'''

In [5]:
# execute this cell to load the urls to the plays

clean_urls = ['https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-10_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-1_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-2_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-3_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-4_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-5_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-6_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-7_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-8_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-French-9_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-10_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-1_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-2_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-3_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-4_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-5_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-6_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-7_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-8_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-German-9_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-10_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-1_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-2_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-3_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-4_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-5_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-6_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-7_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-8_indented.xml',
'https://raw.githubusercontent.com/lucagiovannini7/artificial-dramas/main/ezdrama_format/gpt/xmls/GPT35-Russian-9_indented.xml']

### Iterate over list of URLs to add them to the corpus

In [8]:
# testing 'playname' construction
for url in clean_urls[:3]:
    r = requests.get(url)
    #get only the text from the response and encode it in UTF-8 (important!)
    tei = r.text.encode('utf-8')
    playname = url[96:].split(".xml")[0]
    playname = playname.replace('/','')
    playname = playname.replace('35','3.5')
    playname = playname.replace('_indented','')
    print(playname)

GPT3.5-French-10
GPT3.5-French-1
GPT3.5-French-2


In [ ]:
test_urls = ['https://raw.githubusercontent.com/lucagiovannini7/baroque-networks/main/dracor-xmls/de-dracor/gryphius-cardenio-und-celinde.xml']

In [10]:
# load all plays
counter = 1
for url in clean_urls:
    r = requests.get(url)
    #get only the text from the response and encode it in UTF-8 (important!)
    tei = r.text.encode('utf-8')
    playname = url[96:].split(".xml")[0]
    playname = playname.replace('/','')
    playname = playname.replace('35','3.5')
    playname = playname.replace('_indented','')
    playname = playname.replace('%2B','-')    
    print("Processing", playname)
    #construct the URL to use in the PUT request:
    put_request_url = "http://localhost:8088/api/corpora/" + "ai" + "/play/" + playname + "/tei"
    print("Loading", put_request_url, ". Processed: ", counter, "/30")
    counter += 1
    headers = {'Content-Type': 'application/xml'}
    #loads play
    r = requests.put(put_request_url, data=tei, headers=headers, auth=HTTPBasicAuth(usr, pwd))

Processing GPT3.5-French-10
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-10/tei . Processed:  1 /30
Processing GPT3.5-French-1
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-1/tei . Processed:  2 /30
Processing GPT3.5-French-2
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-2/tei . Processed:  3 /30
Processing GPT3.5-French-3
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-3/tei . Processed:  4 /30
Processing GPT3.5-French-4
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-4/tei . Processed:  5 /30
Processing GPT3.5-French-5
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-5/tei . Processed:  6 /30
Processing GPT3.5-French-6
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-6/tei . Processed:  7 /30
Processing GPT3.5-French-7
Loading http://localhost:8088/api/corpora/ai/play/GPT3.5-French-7/tei . Processed:  8 /30
Processing GPT3.5-French-8
Loading http://localhost:8088/api/c